In [3]:
import k3d 
import numpy as np 
import igl 

In [2]:
import os 
import sys
os.system("pwd")

/Users/timurkesdogan/Desktop/simulation/project/bowling-simulation/src


0

In [3]:
v, _, _, f, _, _ = igl.read_obj("../assets/simple_pin.obj")

  o Cube


In [6]:
v + np.array([1,0,0])

array([[ 0.411534, -1.      ,  0.626688],
       [ 0.520772,  1.683213,  0.442368],
       [ 0.417647, -1.      , -0.518943],
       [ 0.576305,  1.677179, -0.280986],
       [ 1.726965, -1.      ,  0.637821],
       [ 1.23411 ,  1.660197,  0.267153],
       [ 1.595002, -1.      , -0.504613],
       [ 1.21051 ,  1.631795, -0.345299]])

In [4]:
plot = k3d.plot()
plot += k3d.mesh(v, f)
plot.display()

/Users/timurkesdogan/miniconda3/envs/foo/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/Users/timurkesdogan/miniconda3/envs/foo/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [50]:
v = np.array([[0,0,0], [1,0,0], [1,1,0], [0,0,1]], dtype = float)
f = np.array([[0,1,2], [0,2,3], [0,3,1], [1,3,2]], dtype = int  )
q = np.array([[5.0,0,0], [0.2,0.1,0.2]], dtype = float)
P, I, C, N = igl.signed_distance(q, v, f, True)
N

array([[-0.77345908,  0.54691816,  0.32037724],
       [ 0.70710678, -0.70710678,  0.        ]])

In [44]:
j = np.array([1,3,2], dtype = int)
g = np.array([0.5,2,1])
np.sort(g, order=j)

ValueError: Cannot specify order when the array has no fields.

In [ ]:
# S #P list of smallest signed distances
# I #P list of facet indices corresponding to smallest distances
# C #P by 3 list of closest points

In [69]:
import igl
import numpy as np
import polyscope as ps
import k3d
import copy
from utils import Triangle, Object, Collision



object_list = []

# load bowling ball from obj file place in field
v, _, _, f, _, _ = igl.read_obj("../assets/simple_ball.obj")
v = v + np.array([-1, 0, 0])

  o Sphere


In [70]:

object_list.append(Object(v, f))
indices_ball = len(v)


# create global faces and vertices np.arrays
vertices = np.array(v)
faces = np.array(f)

# add pins to the same mesh in this array
v, _, _, f, _, _ = igl.read_obj("../assets/simple_pin.obj")
for i in range(2):
    for j in range(-i, i + 1):
        faces = np.concatenate((faces, f + len(vertices)), axis=0)
        vertices = np.concatenate(
            (vertices, (v + np.array([i * 1.5, 0, 0]) + np.array([0, 0, j * 1.5]))),
            axis=0,
        )
        object_list.append(
            Object(v + np.array([i * 1.5, 0, 0]) + np.array([0, 0, j * 1.5]), f)
        )

# register complete mesh w/ polyscope
ps.init()
mesh = ps.register_surface_mesh("everything", vertices, faces)

# intial all objects as static, except for ball, which is given initial velocity
# slowly approaching pins
initial_velocities = np.zeros(vertices.shape)
initial_velocities[:indices_ball] = np.array([1, 0, 0])

masses = np.ones(len(vertices))
external_forces = np.zeros(vertices.shape)

  o Cube


In [71]:
object_mask = []
vertex_counter = face_counter = 0
object_offsets = []

for object in object_list:
    vertex_mask = np.zeros(len(vertices), dtype=bool)
    face_mask = np.zeros(len(faces), dtype=bool)

    vertex_mask[vertex_counter : vertex_counter + len(object.v)] = True
    face_mask[face_counter : face_counter + len(object.f)] = True
    object_offsets.append((vertex_counter, face_counter))
    vertex_counter += len(object.v)
    face_counter += len(object.f)

    object_mask.append((vertex_mask, face_mask))

In [72]:
i = 0; object = object_list[i]
vertices_i = vertices[~object_mask[i][0]]
faces_i = copy.deepcopy(faces)
faces_i[object_offsets[i][1] :] = faces_i[object_offsets[i][1]:] - len(object.v)
faces_i = faces_i[~object_mask[i][1]]

print(vertices_i.shape, "vs ", vertices.shape)
print(faces_i.shape)

print("min: ", faces_i.min(), "vs ", faces.min())
print("max: ", faces_i.max(), "vs ", faces.max())
print(object.v.shape)

(32, 3) vs  (82, 3)
(48, 3)
min:  0 vs  0
max:  31 vs  81
(50, 3)


In [73]:
plot = k3d.plot()
plot += k3d.mesh(object.v, object.f, color=0xff0000)
plot += k3d.mesh(vertices_i, faces_i, color=0x00ff00)
plot.display()

/home/timur/miniconda3/envs/cloth/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/timur/miniconda3/envs/cloth/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [96]:
S, I, C, N = igl.signed_distance(object.v, vertices_i, faces_i, True)
collision_mask = S < 0

In [97]:
collision_indices = np.arange(len(object.v))[collision_mask]
C = C[collision_mask]
N = N[collision_mask]
S = S[collision_mask]
I = I[collision_mask]

order = np.argsort(S)

collision_indices = collision_indices[order]
C = C[order]
N = N[order]
I = I[order]

I_mask = I >= object_offsets[i][1]
I[I_mask] = I[I_mask] + object.f.shape[0]

In [98]:
# S #P list of smallest signed distances
# I #P list of facet indices corresponding to smallest distances
# C #P by 3 list of closest points
object_offsets[i][1]
I_mask

array([ True,  True,  True,  True,  True,  True,  True])

In [101]:
collision_indices+ object_offsets[i][0]

array([15, 11, 16, 14, 13, 10,  9])